In [5]:
import json
# import os
import xml.etree.ElementTree as ET

# go at root of the project
# os.chdir('/path/to/skillcorner-event-synchronization')

from event_synchronization.event_synchro_manager import EventSynchronizationManager
from event_synchronization.events.opta import OptaEvents
from tools.run_opta import get_event_format
from skillcorner.client import SkillcornerClient


skc_client = SkillcornerClient(
    username='USERNAME',
    password='PASSWORD'
)

# Load data

In [2]:
MATCH_ID = 45623

# load Skillcorner match_data
match_data = skc_client.get_match(MATCH_ID)

# load Skillcorner tracking_data
tracking_data = skc_client.get_match_tracking_data(MATCH_ID, params={'data_version':'3'})

# load Opta data
opta_events_path = '/path/to/opta_events.xml'
opta_match_data_path = '/path/to/opta_match_data.xml'
event_format = get_event_format(opta_events_path)

if event_format == 'json':
    # load raw_opta_events
    with open(opta_events_path) as f:
        raw_opta_events = json.load(f)

    # load opta_match_data
    with open(opta_match_data_path) as f:
        opta_match_data = json.load(f)
else:
    raw_opta_events = ET.parse(opta_events_path)
    opta_match_data = ET.parse(opta_match_data_path)

# Run Event Synchronization Process

In [3]:
# standardize events
opta_events = OptaEvents(raw_opta_events, opta_match_data, match_data, format=event_format)

# apply the whole event synchronization process
report_by_event, report_by_event_type, freeze_frame_format = EventSynchronizationManager(
    tracking_data, match_data, opta_events
).apply_synchronization_process()